[Link to colab](https://colab.research.google.com/github/lsmin0152/cheb301/blob/main/notebooks/CHEB301_F25_05_MD_HOOMD-Blue.ipynb)
# Running MD simulation via HOOMD-Blue

We will learn how to run a MD simulation using HOOMD-Blue particle simulation package. \
Documentation: https://hoomd-blue.readthedocs.io/en/v5.4.0/ \
Github: https://github.com/glotzerlab/hoomd-blue

HOOMD-blue is a Python package that runs simulations of particle systems on CPUs and GPUs. It performs hard particle Monte Carlo simulations of a variety of shape classes and molecular dynamics simulations of particles with a range of pair, bond, angle, and other potentials. Many features are targeted at the soft matter research community, though the code is general and capable of many types of particle simulations.

In [ ]:
# Install dependencies
!apt-get update
!apt install -y cmake git
!apt-get install -y libeigen3-dev
!pip install --upgrade pip
!pip install eigen ninja numpy pybind11==2.12

# Pull source code
!rm -rf hoomd-blue
!git clone --recursive https://github.com/glotzerlab/hoomd-blue.git hoomd-blue


In [ ]:
# Pull source code and build

%cd ./hoomd-blue
!cmake -B build \
-DBUILD_METAL=off \
-DBUILD_MPCD=off \
-DBUILD_TESTING=off \
-Dpybind11_DIR=/usr/local/lib/python3.12/dist-packages/pybind11/share/cmake/pybind11 \
-DEigen3_DIR=/usr/lib/cmake/eigen3 \
-S . -GNinja

In [ ]:
# Compiling the source code

%cd ./build
!ninja -j 2

## Create the simulation and initial state

In [ ]:
import hoomd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def init_coord_bcc(a,n):
  # a = lattice parameter
  # n = number of unit cells in each direction

  # basis atoms of bcc in fractional coordinates
  basis = np.array([
      [0.0, 0.0, 0.0],
      [0.5, 0.5, 0.5]
  ])

  coords = []

  for i in range(n):
      for j in range(n):
          for k in range(n):
              cell_origin = np.array([i, j, k], dtype=float)
              for b in basis:
                  coords.append((cell_origin + b) * a)

  coords = np.array(coords)
  box = np.array([8*a, 8*a, 8*a])
  return box, coords

box, positions = init_coord_bcc(1.5,4)

print(box)
print(np.max(positions))

In [ ]:
# Load a function for visualization
def plot_particles(coords):

  # 3D plot
  fig = plt.figure(figsize=(6,6))
  ax = fig.add_subplot(111, projection='3d')

  x = coords[:,0]
  y = coords[:,1]
  z = coords[:,2]

  ax.scatter(x, y, z, c='blue', marker='o', s=40)

  ax.set_xlabel('X')
  ax.set_ylabel('Y')
  ax.set_zlabel('Z')
  ax.set_title('3D particle positions')

  plt.show()


In [ ]:
# Initialization

cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=2)

snapshot = hoomd.Snapshot()
snapshot.particles.N = len(positions)
snapshot.particles.position[:] = positions
snapshot.particles.types = ['A']
snapshot.configuration.box = [box[0], box[1], box[2], 0, 0, 0]

sim.create_state_from_snapshot(snapshot)


In [ ]:
# Visualize the initial state
plot_particles(positions)

## Molecular dynamics classes

The MD integrator numerically integrates the equations of motion:

In [ ]:
integrator = hoomd.md.Integrator(dt=0.005)

You need a neighbor list to compte pairwise interactions:

In [ ]:
cell = hoomd.md.nlist.Cell(buffer=0.4)

Compute Lennard-Jones interaction forces:
$$V_{\mathrm{LJ}}(r) = 4 \varepsilon \left[ \left(\frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r}\right)^6 \right]$$

In [ ]:
lj = hoomd.md.pair.LJ(nlist=cell)
lj.params[('A', 'A')] = dict(epsilon=1, sigma=1)

Set `r_cut` to 2.5:

In [ ]:
lj.r_cut[('A', 'A')] = 2.5

Add the `lj` force to the integrator's `forces` list:

In [ ]:
integrator.forces.append(lj)

The integration method sets the specific equations of motion that apply to a subset of particles:

In [ ]:
all_particles = hoomd.filter.All()
kT = 0.5
nvt = hoomd.md.methods.ConstantVolume(thermostat=hoomd.md.methods.thermostats.Bussi(kT=1.5), filter=all_particles)
mttk = hoomd.md.methods.thermostats.MTTK(kT=kT, tau=0.2)


Add `nvt` to the integrator's `methods` list:

In [ ]:
integrator.methods.append(nvt)

Remember to add the integrator to the simulation operations!

In [ ]:
sim.operations += integrator
sim.operations.integrator.methods[0].thermostat = mttk

## Running a molecular dynamics simulation

HOOMD-blue defaults particle velocities to 0:

In [ ]:
print(sim.state.get_snapshot().particles.velocity[0:5, :])

Thermalize the velocities and integration method's degrees of freedom before running.

In [ ]:
sim.state.thermalize_particle_momenta(filter=all_particles, kT=kT)
sim.run(0)
mttk.thermalize_dof()
print(sim.state.get_snapshot().particles.velocity[0:5, :])

Get initial configurations

In [ ]:
pos_0 = sim.state.get_snapshot().particles.position
box_0 = sim.state.get_snapshot().configuration.box

plot_particles(pos_0)

Run the simulation:

In [ ]:
sim.run(10000)

pos_1 = sim.state.get_snapshot().particles.position
box_1 = sim.state.get_snapshot().configuration.box

plot_particles(pos_1)

## Querying thermodynamic quantities

The `ThermodynamicQuantities` class computes properties of the simulation:

In [ ]:
thermo = hoomd.md.compute.ThermodynamicQuantities(filter=all_particles)
sim.operations += thermo

Use `thermo` to get the `potential_energy` of the system:

In [ ]:
print(thermo.potential_energy)
print(thermo.kinetic_energy)
print(thermo.kinetic_temperature)


In [ ]:
# Run again
sim.run(10000)
pos_2 = sim.state.get_snapshot().particles.position
box_2 = sim.state.get_snapshot().configuration.box

print(thermo.potential_energy)
print(thermo.kinetic_energy)
print(thermo.kinetic_temperature)

plot_particles(pos_2)

In [ ]:
# Run again
sim.run(100000)
pos_3 = sim.state.get_snapshot().particles.position
box_3 = sim.state.get_snapshot().configuration.box

print(thermo.potential_energy)
print(thermo.kinetic_energy)
print(thermo.kinetic_temperature)

plot_particles(pos_3)
